
# Simplex Linear Programming — Step-by-Step (Python)

This notebook implements the Simplex algorithm (with **Two-Phase** support for `>=` and `=` constraints) and displays each iteration **step-by-step** with readable tableaus.  
It can solve problems in the form:

- Objective: **maximize** or **minimize** a linear function `c^T x`
- Subject to linear constraints of type `<=`, `>=`, or `=`
- Decision variables `x >= 0`

You can define your own problems by filling in `c`, `A`, `b`, and the constraint types, then call `solve()` to see every step, including entering/leaving variables, pivots, and final solution.



## How to use

1. Edit the problem definition in the **"🧩 Define your LP problem"** cell:
   - `sense`: `"max"` or `"min"`  
   - `c`: coefficients of the objective function (list of length `n`)  
   - `A`: constraint matrix (list of `m` rows each with `n` numbers)  
   - `b`: right-hand sides (length `m`)  
   - `cons`: list of the same length `m` with constraint types from `{ "<=", ">=", "=" }`

2. Run the **Solver** cell to get a **step-by-step** trace and the final answer.

3. Try the ready-made examples at the bottom (including a two-phase case and infeasible/unbounded demonstrations).


two-phase

## Code

In [1]:
from typing import List, Dict, Tuple, Optional
import numpy as np
import pandas as pd

np.set_printoptions(suppress=True)

class SimplexResult:
    def __init__(self, status: str, objective: float, solution: Dict[str, float], iterations: int, steps: List[str]):
        self.status = status              # 'optimal', 'unbounded', 'infeasible'
        self.objective = objective        # optimal objective (if any)
        self.solution = solution          # variable -> value at termination
        self.iterations = iterations      # number of pivots performed
        self.steps = steps                # textual log of steps

def _format_df(tableau: np.ndarray, row_names: List[str], col_names: List[str]) -> pd.DataFrame:
    df = pd.DataFrame(tableau, columns=col_names, index=row_names)
    # round for readability
    return df.round(6)

class TwoPhaseSimplex:
    def __init__(self, sense: str, c: List[float], A: List[List[float]], b: List[float], cons: List[str], tol: float=1e-9, bland: bool=True):
        """
        sense: 'max' or 'min'
        c: objective coefficients (length n)
        A: constraint matrix (m x n)
        b: RHS (length m)
        cons: constraint sense for each row: '<=', '>=', or '='
        tol: tolerance for numerical comparisons
        bland: use Bland's rule to avoid cycling (tie-breaking by lowest-index variable)
        """
        self.sense = sense.lower()
        assert self.sense in ('max', 'min')
        self.c = np.array(c, dtype=float)
        self.A = np.array(A, dtype=float)
        self.b = np.array(b, dtype=float)
        self.cons = cons
        self.tol = float(tol)
        self.bland = bland

        m, n = self.A.shape
        assert len(self.c) == n, "c and A have incompatible sizes"
        assert len(self.b) == m and len(self.cons) == m, "b/cons length must match number of constraints"

        # If minimization, convert to maximization by negating c.
        self.orig_c = self.c.copy()
        if self.sense == 'min':
            self.c = -self.c
            self.sense = 'max'  # internal: always treat as maximization

        # Normalize constraints so that b >= 0 by multiplying by -1 if needed
        self.An = self.A.copy()
        self.bn = self.b.copy()
        self.cons_norm = list(self.cons)
        for i in range(m):
            if self.bn[i] < -self.tol:
                # multiply row by -1 and flip sense
                self.An[i, :] *= -1
                self.bn[i] *= -1
                if self.cons_norm[i] == '<=':
                    self.cons_norm[i] = '>='
                elif self.cons_norm[i] == '>=':
                    self.cons_norm[i] = '<='
                # '=' stays '='

        self.m = m
        self.n = n

        # Build full column structure: [x vars] + [slacks/surplus] + [artificials]
        self.var_names: List[str] = [f"x{j+1}" for j in range(n)]
        self.slack_idx = []
        self.surplus_idx = []
        self.artificial_idx = []
        self.basic_vars = [-1]*m     # column index of basic variable in each row

        cols = []
        # start with original variables
        cols.extend([self.An[:, j] for j in range(n)])

        # Add slack/surplus/artificials per row
        for i in range(m):
            sense = self.cons_norm[i]
            col_slack = np.zeros(m)
            col_surplus = np.zeros(m)
            col_art = np.zeros(m)
            if sense == '<=':
                # add slack +1
                col_slack[i] = 1.0
                self.slack_idx.append(len(self.var_names))
                self.var_names.append(f"s{i+1}")
                cols.append(col_slack)
                # slack is initial basic var
                self.basic_vars[i] = self.var_names.index(f"s{i+1}")
            elif sense == '>=':
                # add surplus -1 and artificial +1
                col_surplus[i] = -1.0
                self.surplus_idx.append(len(self.var_names))
                self.var_names.append(f"t{i+1}")
                cols.append(col_surplus)
                col_art[i] = 1.0
                self.artificial_idx.append(len(self.var_names))
                self.var_names.append(f"a{i+1}")
                cols.append(col_art)
                # artificial is initial basic var
                self.basic_vars[i] = self.var_names.index(f"a{i+1}")
            elif sense == '=':
                # artificial +1
                col_art[i] = 1.0
                self.artificial_idx.append(len(self.var_names))
                self.var_names.append(f"a{i+1}")
                cols.append(col_art)
                # artificial is initial basic var
                self.basic_vars[i] = self.var_names.index(f"a{i+1}")
            else:
                raise ValueError("Constraint sense must be one of '<=', '>=', '='")

        self.A_full = np.column_stack(cols)  # shape m x total_vars
        self.total_vars = self.A_full.shape[1]

        # Vector of costs for original objective across all current variables (x, slack/surplus 0, artificial 0)
        self.c_full = np.zeros(self.total_vars)
        self.c_full[:self.n] = self.c  # only original x have cost

        # Phase I costs: 0 for non-artificial, -1 for artificial (maximize -sum(a))
        self.c_phase1 = np.zeros(self.total_vars)
        for idx in self.artificial_idx:
            self.c_phase1[idx] = -1.0

        # Build initial tableau for Phase I as [A | b] with last row Cj - Zj
        self.tableau = np.zeros((self.m + 1, self.total_vars + 1))
        self.tableau[:self.m, :self.total_vars] = self.A_full
        self.tableau[:self.m, -1] = self.bn

        # Build Phase I objective row: C - Z with basic = artificials where present
        # Start with C = c_phase1
        self.tableau[-1, :self.total_vars] = self.c_phase1.copy()
        self.tableau[-1, -1] = 0.0
        # Subtract sum(c_B * row) to form C - Z (note: c_B for slack-only rows is 0)
        for i in range(self.m):
            bcol = self.basic_vars[i]
            if bcol in self.artificial_idx:
                c_Bi = self.c_phase1[bcol]  # = -1
                # C - Z := C - c_Bi * row_i
                # Since c_Bi = -1, this is C - (-1)*row = C + row
                self.tableau[-1, :] -= c_Bi * self.tableau[i, :]

        self.steps = []  # log strings
        self.iterations = 0

    def _current_objective_value(self, row: Optional[np.ndarray]=None) -> float:
        if row is None:
            row = self.tableau[-1, :]
        # C - Z tableau stores RHS at bottom as -Z (because RHS: 0 - c_B^T b = -Z)
        return -row[-1]

    def _choose_entering(self, phase: int) -> Optional[int]:
        # Choose entering column: largest positive (C - Z) coefficient
        obj_row = self.tableau[-1, :-1]
        candidates = np.where(obj_row > self.tol)[0]
        if candidates.size == 0:
            return None
        if self.bland:
            return int(candidates.min())
        # Otherwise pick argmax
        return int(candidates[np.argmax(obj_row[candidates])])

    def _choose_leaving(self, enter_col: int) -> Optional[int]:
        col = self.tableau[:self.m, enter_col]
        rhs = self.tableau[:self.m, -1]
        mask = col > self.tol
        if not mask.any():
            return None  # unbounded
        ratios = np.where(mask, rhs / col, np.inf)
        # Tie-break with Bland: pick smallest row index among min ratios
        min_val = ratios[mask].min()
        candidate_rows = np.where(np.isclose(ratios, min_val, atol=1e-12))[0]
        return int(candidate_rows.min())

    def _pivot(self, row: int, col: int):
        """
        Performs a pivot operation on the tableau.
        The pivot is at the intersection of `row` and `col`.
        """
        pivot_element = self.tableau[row, col]
        # Log the row operation for the pivot row
        self.steps.append(
            f"   Row R{row+1} divided by pivot element {pivot_element:.6f} to make it 1."
        )
        self.tableau[row, :] = self.tableau[row, :] / pivot_element

        # Eliminate other row entries
        for r in range(self.m + 1):
            if r == row:
                continue
            factor = self.tableau[r, col]
            if abs(factor) > self.tol:
                # Log the row operation for non-pivot rows
                self.steps.append(
                    f"   Row R{r+1} updated: R{r+1} - ({factor:.6f}) * R{row+1}"
                )
                self.tableau[r, :] -= factor * self.tableau[row, :]

        # Update the basic variable for the pivoted row
        self.basic_vars[row] = col

    def _log_tableau(self, title: str):
        row_names = [self.var_names[self.basic_vars[i]] if self.basic_vars[i] >= 0 else f"row{i+1}" for i in range(self.m)] + ["C - Z"]
        col_names = self.var_names + ["RHS"]
        df = _format_df(self.tableau, row_names, col_names)
        # Store a tuple: (title, DataFrame)
        self.steps.append((title, df))


    def _phase(self, phase_num: int, cvec: np.ndarray, drop_artificial: bool=False) -> Tuple[str, float]:
        # Reset bottom row to C - Z for the given cost vector cvec.
        self.tableau[-1, :] = 0.0
        self.tableau[-1, :self.total_vars] = cvec.copy()
        for i in range(self.m):
            bcol = self.basic_vars[i]
            if bcol >= 0:
                self.tableau[-1, :] -= cvec[bcol] * self.tableau[i, :]

        self._log_tableau(f"Phase {phase_num} — Initial C - Z row")        

        while True:
            enter = self._choose_entering(phase_num)
            if enter is None:
                # Optimal for this phase
                z = self._current_objective_value()
                self._log_tableau(f"Phase {phase_num} — Optimal (no positive C-Z). Objective = {z:.6f}")
                return ("optimal", z)

            leave = self._choose_leaving(enter)
            if leave is None:
                self._log_tableau(f"Phase {phase_num} — Unbounded in entering {self.var_names[enter]}")
                return ("unbounded", np.inf)

            self.iterations += 1
            self.steps.append(f"Iteration {self.iterations} (Phase {phase_num}): Enter = {self.var_names[enter]}, Leave = {self.var_names[self.basic_vars[leave]]}")
            self._pivot(leave, enter)
            self._log_tableau(f"After pivot #{self.iterations}")

    def solve(self, show_steps: bool=True) -> SimplexResult:
        # Phase I (only if there are artificials)
        if len(self.artificial_idx) > 0:
            status1, z1 = self._phase(1, self.c_phase1)
            if status1 != "optimal" or z1 < -self.tol:
                # If unbounded in phase I or objective < 0 (shouldn't happen with -sum(a)), report infeasible
                return SimplexResult("infeasible", float("nan"), {}, self.iterations, self.steps)
            # Check feasibility: z1 should be 0 (within tol)
            if abs(z1) > 1e-7:
                self.steps.append(f"Phase 1 objective is {z1:.6f} > 0 → Infeasible.")
                return SimplexResult("infeasible", float("nan"), {}, self.iterations, self.steps)

            # Remove artificial columns; but first try to pivot out any artificial still basic
            for i in range(self.m):
                bcol = self.basic_vars[i]
                if bcol in self.artificial_idx:
                    # Try to pivot to a non-artificial column with non-zero coeff
                    for j in range(self.total_vars):
                        if j in self.artificial_idx:
                            continue
                        if abs(self.tableau[i, j]) > self.tol:
                            self._pivot(i, j)
                            break
                    else:
                        # Entire row may be redundant (zero row with RHS 0). Keep but it's okay to proceed.
                        pass

            # Now drop artificial columns from tableau & names
            keep_cols = [j for j in range(self.total_vars) if j not in self.artificial_idx]
            # Rebuild structures
            self.A_full = self.A_full[:, keep_cols]
            self.var_names = [self.var_names[j] for j in keep_cols]
            self.total_vars = len(keep_cols)
            self.c_full = np.zeros(self.total_vars)
            self.c_full[:self.n] = self.c[:self.n]  # original costs for x vars
            # Slice tableau
            self.tableau = np.column_stack([self.tableau[:, keep_cols], self.tableau[:, -1]])
            # Update basic var indices to new column indexing
            old_to_new = {old: new for new, old in enumerate(keep_cols)}
            for i in range(self.m):
                if self.basic_vars[i] in old_to_new:
                    self.basic_vars[i] = old_to_new[self.basic_vars[i]]
                else:
                    # If a basic var got removed (shouldn't after pivoting), mark as -1
                    self.basic_vars[i] = -1

        # Phase II (optimize original objective)
        status2, z2 = self._phase(2, self.c_full)
        if status2 == "unbounded":
            return SimplexResult("unbounded", float("inf"), {}, self.iterations, self.steps)

        # Extract solution: basic RHS values, nonbasic 0
        sol = {name: 0.0 for name in self.var_names}
        for i in range(self.m):
            bcol = self.basic_vars[i]
            if bcol >= 0:
                sol[self.var_names[bcol]] = self.tableau[i, -1]

        # Report only original decision variables x1..xn
        x_solution = {f"x{j+1}": sol.get(f"x{j+1}", 0.0) for j in range(self.n)}
        objective_value = z2  # since we used max internally

        # If original was 'min', flip sign back
        if (self.orig_c != self.c[:self.n]).any():
            # minimization: objective_value_max = z2 = -min_value
            objective_value = -z2

        return SimplexResult("optimal", objective_value, x_solution, self.iterations, self.steps)


BigM

In [2]:
from typing import List, Dict, Tuple, Optional
import numpy as np
import pandas as pd

np.set_printoptions(suppress=True)

class SimplexResult:
    def __init__(self, status: str, objective: float, solution: Dict[str, float], iterations: int, steps: List[str]):
        self.status = status                # 'optimal', 'unbounded', 'infeasible'
        self.objective = objective          # optimal objective (if any)
        self.solution = solution            # variable -> value at termination
        self.iterations = iterations        # number of pivots performed
        self.steps = steps                  # textual log of steps

def _format_df(tableau: np.ndarray, row_names: List[str], col_names: List[str]) -> pd.DataFrame:
    df = pd.DataFrame(tableau, columns=col_names, index=row_names)
    # round for readability
    return df.round(6)

class BigMSimplex:
    def __init__(self, sense: str, c: List[float], A: List[List[float]], b: List[float], cons: List[str], M: float=1e6, tol: float=1e-9, bland: bool=True):
        self.sense = sense.lower()
        assert self.sense in ('max', 'min')
        self.c = np.array(c, dtype=float)
        self.A = np.array(A, dtype=float)
        self.b = np.array(b, dtype=float)
        self.cons = cons
        self.M = float(M)
        self.tol = float(tol)
        self.bland = bland

        m, n = self.A.shape
        self.m = m
        self.n = n

        # Normalize constraints so that b >= 0
        self.An = self.A.copy()
        self.bn = self.b.copy()
        self.cons_norm = list(self.cons)
        for i in range(m):
            if self.bn[i] < -self.tol:
                self.An[i, :] *= -1
                self.bn[i] *= -1
                if self.cons_norm[i] == '<=':
                    self.cons_norm[i] = '>='
                elif self.cons_norm[i] == '>=':
                    self.cons_norm[i] = '<='

        # Build full column structure
        self.var_names: List[str] = [f"x{j+1}" for j in range(n)]
        self.artificial_idx = []
        self.basic_vars = [-1]*m

        cols = []
        c_full_list = []
        for j in range(n):
            cols.append(self.An[:, j])
            c_full_list.append(self.c[j])

        for i in range(m):
            sense = self.cons_norm[i]
            if sense == '<=':
                col_slack = np.zeros(m)
                col_slack[i] = 1.0
                cols.append(col_slack)
                self.var_names.append(f"s{i+1}")
                c_full_list.append(0.0)
                self.basic_vars[i] = self.var_names.index(f"s{i+1}")
            elif sense == '>=':
                col_surplus = np.zeros(m)
                col_surplus[i] = -1.0
                cols.append(col_surplus)
                self.var_names.append(f"t{i+1}")
                c_full_list.append(0.0)
                
                col_art = np.zeros(m)
                col_art[i] = 1.0
                cols.append(col_art)
                self.var_names.append(f"a{i+1}")
                self.artificial_idx.append(len(c_full_list))
                c_full_list.append(-self.M if self.sense == 'max' else self.M)
                self.basic_vars[i] = self.var_names.index(f"a{i+1}")
            elif sense == '=':
                col_art = np.zeros(m)
                col_art[i] = 1.0
                cols.append(col_art)
                self.var_names.append(f"a{i+1}")
                self.artificial_idx.append(len(c_full_list))
                c_full_list.append(-self.M if self.sense == 'max' else self.M)
                self.basic_vars[i] = self.var_names.index(f"a{i+1}")
            else:
                raise ValueError("Constraint sense must be one of '<=', '>=', '='")

        self.A_full = np.column_stack(cols)
        self.total_vars = self.A_full.shape[1]

        # Final C vector for the tableau, converted to maximization
        self.c_full = np.array(c_full_list, dtype=float)
        if self.sense == 'min':
            self.c_full = -self.c_full
            self.orig_sense = 'min'
        else:
            self.orig_sense = 'max'

        # Build initial tableau
        self.tableau = np.zeros((self.m + 1, self.total_vars + 1))
        self.tableau[:self.m, :self.total_vars] = self.A_full
        self.tableau[:self.m, -1] = self.bn

        # Build initial C - Z row
        self.tableau[-1, :self.total_vars] = self.c_full.copy()
        for i in range(self.m):
            bcol = self.basic_vars[i]
            if bcol >= 0:
                self.tableau[-1, :] -= self.c_full[bcol] * self.tableau[i, :]

        self.steps = []
        self.iterations = 0

    def _current_objective_value(self) -> float:
        return -self.tableau[-1, -1]

    def _choose_entering(self) -> Optional[int]:
        obj_row = self.tableau[-1, :-1]
        candidates = np.where(obj_row > self.tol)[0]
        if candidates.size == 0:
            return None
        if self.bland:
            return int(candidates.min())
        return int(candidates[np.argmax(obj_row[candidates])])

    def _choose_leaving(self, enter_col: int) -> Optional[int]:
        col = self.tableau[:self.m, enter_col]
        rhs = self.tableau[:self.m, -1]
        mask = col > self.tol
        if not mask.any():
            return None
        ratios = np.where(mask, rhs / col, np.inf)
        min_val = ratios[mask].min()
        candidate_rows = np.where(np.isclose(ratios, min_val, atol=1e-12))[0]
        return int(candidate_rows.min())

    def _pivot(self, row: int, col: int):
        """
        Performs a pivot operation on the tableau.
        The pivot is at the intersection of `row` and `col`.
        """
        pivot_element = self.tableau[row, col]
        # Log the row operation for the pivot row
        self.steps.append(
            f"   Row R{row+1} divided by pivot element {pivot_element:.6f} to make it 1."
        )
        self.tableau[row, :] = self.tableau[row, :] / pivot_element

        # Eliminate other row entries
        for r in range(self.m + 1):
            if r == row:
                continue
            factor = self.tableau[r, col]
            if abs(factor) > self.tol:
                # Log the row operation for non-pivot rows
                self.steps.append(
                    f"   Row R{r+1} updated: R{r+1} - ({factor:.6f}) * R{row+1}"
                )
                self.tableau[r, :] -= factor * self.tableau[row, :]

        # Update the basic variable for the pivoted row
        self.basic_vars[row] = col

    def _log_tableau(self, title: str):
        row_names = [self.var_names[self.basic_vars[i]] if self.basic_vars[i] >= 0 else f"row{i+1}" for i in range(self.m)] + ["C - Z"]
        col_names = self.var_names + ["RHS"]
        df = _format_df(self.tableau, row_names, col_names)
        self.steps.append((title, df))

    def solve(self, show_steps: bool=True) -> SimplexResult:
        self._log_tableau("Initial Tableau (Big M Method)")

        while True:
            enter = self._choose_entering()
            if enter is None:
                break
            leave = self._choose_leaving(enter)
            if leave is None:
                return SimplexResult("unbounded", float("inf"), {}, self.iterations, self.steps)
            
            self.iterations += 1
            self.steps.append(f"Iteration {self.iterations}: Enter = {self.var_names[enter]}, Leave = {self.var_names[self.basic_vars[leave]]}")
            self._pivot(leave, enter)
            self._log_tableau(f"After pivot #{self.iterations}")

        # Final checks
        for i in range(self.m):
            bcol = self.basic_vars[i]
            if bcol in self.artificial_idx:
                if self.tableau[i, -1] > self.tol:
                    self.steps.append("Final solution contains an artificial variable with a positive value. Infeasible.")
                    return SimplexResult("infeasible", float("nan"), {}, self.iterations, self.steps)

        # Extract solution
        sol = {name: 0.0 for name in self.var_names}
        for i in range(self.m):
            bcol = self.basic_vars[i]
            if bcol >= 0:
                sol[self.var_names[bcol]] = self.tableau[i, -1]

        x_solution = {f"x{j+1}": sol.get(f"x{j+1}", 0.0) for j in range(self.n)}
        objective_value = self._current_objective_value()

        if self.orig_sense == 'min':
            objective_value = -objective_value

        return SimplexResult("optimal", objective_value, x_solution, self.iterations, self.steps)


In [3]:
def run_and_show(sense, c, A, b, cons, Mode, M):
    if Mode == 'BigM':
        solver = BigMSimplex(sense=sense, c=c, A=A, b=b, cons=cons, M=M, bland=True)
    else:
        solver = TwoPhaseSimplex(sense=sense, c=c, A=A, b=b, cons=cons, bland=True)
    result = solver.solve(show_steps=True)
    print("\n=== Mode ===")
    print("Mode:", Mode)

    from IPython.display import Markdown, display
    # Show step logs
    for step in result.steps:
        if isinstance(step, tuple):
            title, df = step
            display(Markdown(f"### {title}"))
            display(df)
        else:
            display(Markdown(step))


    print("\n=== RESULT ===")
    print("Status:", result.status)
    print("Objective value:", result.objective)
    print("Solution:", result.solution)
    print("Iterations:", result.iterations)


## 🧩 Define your LP problem

### twoPhase

In [4]:
# Example (change these to your own problem)
sense = 'min'  # 'max' or 'min'
c = [48, 52, 65]     # maximize 3x1 + 2x2
A = [
    [4, 5, 3],
    [2, 7, 6]
]
b = [50, 30]
cons = ['>=', '>=']
Mode = 'TwoPhase'

run_and_show(sense, c, A, b, cons, Mode, M=1e5)


=== Mode ===
Mode: TwoPhase


### Phase 1 — Initial C - Z row

,x1,x2,x3,t1,a1,t2,a2,RHS
a1,4.0,5.0,3.0,-1.0,1.0,0.0,0.0,50.0
a2,2.0,7.0,6.0,0.0,0.0,-1.0,1.0,30.0
C - Z,6.0,12.0,9.0,-1.0,0.0,-1.0,0.0,80.0


Iteration 1 (Phase 1): Enter = x1, Leave = a1

   Row R1 divided by pivot element 4.000000 to make it 1.

...

### After pivot #3

,x1,x2,x3,t1,t2,RHS
t2,3.6,0.0,-1.8,-1.4,1.0,40.0
x2,0.8,1.0,0.6,-0.2,0.0,10.0
C - Z,-6.4,0.0,-33.8,-10.4,0.0,520.0


### Phase 2 — Optimal (no positive C-Z). Objective = -520.000000

,x1,x2,x3,t1,t2,RHS
t2,3.6,0.0,-1.8,-1.4,1.0,40.0
x2,0.8,1.0,0.6,-0.2,0.0,10.0
C - Z,-6.4,0.0,-33.8,-10.4,0.0,520.0



=== RESULT ===
Status: optimal
Objective value: 520.0
Solution: {'x1': 0.0, 'x2': np.float64(10.0), 'x3': 0.0}
Iterations: 3


### BigM

In [5]:
# Example (change these to your own problem)
sense = 'min'  # 'max' or 'min'
c = [0.4, 0.5]     # maximize 3x1 + 2x2
A = [
    [0.3, 0.1],
    [0.5, 0.5],
    [0.6, 0.4]
]
b = [2.7, 6, 6]
cons = ['<=', '=', '>=']
Mode = 'BigM'  # or 'TwoPhase'

run_and_show(sense, c, A, b, cons, Mode, M=1e5)


=== Mode ===
Mode: BigM


### Initial Tableau (Big M Method)

,x1,x2,s1,a2,t3,a3,RHS
s1,0.3,0.1,1.0,0.0,0.0,0.0,2.7
a2,0.5,0.5,0.0,1.0,0.0,0.0,6.0
a3,0.6,0.4,0.0,0.0,-1.0,1.0,6.0
C - Z,109999.6,89999.5,0.0,0.0,-100000.0,0.0,1200000.0


Iteration 1: Enter = x1, Leave = s1

   Row R1 divided by pivot element 0.300000 to make it 1.

...

   Row R3 updated: R3 - (5.000000) * R2

   Row R4 updated: R4 - (0.500000) * R2

### After pivot #4

,x1,x2,s1,a2,t3,a3,RHS
x1,1.0,0.0,5.0,-1.0,0.0,0.0,7.50
t3,0.0,0.0,1.0,0.6,1.0,-1.0,0.30
x2,0.0,1.0,-5.0,3.0,0.0,0.0,4.50
C - Z,0.0,0.0,-0.5,-99998.9,0.0,-100000.0,5.25



=== RESULT ===
Status: optimal
Objective value: 5.249999999978172
Solution: {'x1': np.float64(7.500000000000003), 'x2': np.float64(4.499999999999997)}
Iterations: 4


## OR2

### GOA ผลิตสี

In [ ]:
# Example (change these to your own problem)
sense = 'max'  # 'max' or 'min'
c = [5, 4]     # maximize 3x1 + 2x2
A = [
    [6, 4],
    [1, 2],
    [0, 1],
    [-1,1]
]
b = [24, 6, 2, 1]
cons = ['<=', '<=', '<=', '<=']

Mode = 'TwoPhase'

run_and_show(sense, c, A, b, cons, Mode, M=1e5)


C:\Users\Admin\AppData\Local\Temp\ipykernel_2104\2753902525.py:172: RuntimeWarning: divide by zero encountered in divide
  ratios = np.where(mask, rhs / col, np.inf)


### Phase 2 — Initial C - Z row

,x1,x2,s1,s2,s3,s4,RHS
s1,6.0,4.0,1.0,0.0,0.0,0.0,24.0
s2,1.0,2.0,0.0,1.0,0.0,0.0,6.0
s3,0.0,1.0,0.0,0.0,1.0,0.0,2.0
s4,-1.0,1.0,0.0,0.0,0.0,1.0,1.0
C - Z,5.0,4.0,0.0,0.0,0.0,0.0,0.0


Iteration 1 (Phase 2): Enter = x1, Leave = s1

### After pivot #1

...

### After pivot #2

,x1,x2,s1,s2,s3,s4,RHS
x1,1.0,0.0,0.250,-0.50,0.0,0.0,3.0
x2,0.0,1.0,-0.125,0.75,0.0,0.0,1.5
s3,0.0,0.0,0.125,-0.75,1.0,0.0,0.5
s4,0.0,0.0,0.375,-1.25,0.0,1.0,2.5
C - Z,0.0,0.0,-0.750,-0.50,0.0,0.0,-21.0


### Phase 2 — Optimal (no positive C-Z). Objective = 21.000000

,x1,x2,s1,s2,s3,s4,RHS
x1,1.0,0.0,0.250,-0.50,0.0,0.0,3.0
x2,0.0,1.0,-0.125,0.75,0.0,0.0,1.5
s3,0.0,0.0,0.125,-0.75,1.0,0.0,0.5
s4,0.0,0.0,0.375,-1.25,0.0,1.0,2.5
C - Z,0.0,0.0,-0.750,-0.50,0.0,0.0,-21.0



=== RESULT ===
Status: optimal
Objective value: 21.0
Solution: {'x1': np.float64(3.0), 'x2': np.float64(1.4999999999999998)}
Iterations: 2


### ผลิตอาหารสัตว์(4)

In [ ]:
# Example (change these to your own problem)
sense = 'min'  # 'max' or 'min'
c = [0.3, 0.9]     # maximize 3x1 + 2x2
A = [
    [1, 1],
    [0.21, -0.3],
    [0.03, -0.01]
]
b = [24, 6, 2]
cons = ['>=', '<=', '>=']

Mode = 'TwoPhase'

run_and_show(sense, c, A, b, cons, Mode, M=1e5)


### Phase 1 — Initial C - Z row

,x1,x2,t1,a1,s2,t3,a3,RHS
a1,1.00,1.00,-1.0,1.0,0.0,0.0,0.0,24.0
s2,0.21,-0.30,0.0,0.0,1.0,0.0,0.0,6.0
a3,0.03,-0.01,0.0,0.0,0.0,-1.0,1.0,2.0
C - Z,1.03,0.99,-1.0,0.0,0.0,-1.0,0.0,26.0


Iteration 1 (Phase 1): Enter = x1, Leave = a1

### After pivot #1

,x1,x2,t1,a1,s2,t3,a3,RHS
x1,1.0,1.00,-1.00,1.00,0.0,0.0,0.0,24.00
s2,0.0,-0.51,0.21,-0.21,1.0,0.0,0.0,0.96
a3,0.0,-0.04,0.03,-0.03,0.0,-1.0,1.0,1.28
C - Z,0.0,-0.04,0.03,-1.03,0.0,-1.0,0.0,1.28


...

### Phase 2 — Initial C - Z row

,x1,x2,t1,s2,t3,RHS
x1,1.0,0.0,0.0,-1.449275,-43.478261,78.260870
t1,0.0,0.0,1.0,-5.797101,-73.913043,89.043478
x2,0.0,1.0,0.0,-4.347826,-30.434783,34.782609
C - Z,0.0,0.0,0.0,-4.347826,-40.434783,54.782609


### Phase 2 — Optimal (no positive C-Z). Objective = -54.782609

,x1,x2,t1,s2,t3,RHS
x1,1.0,0.0,0.0,-1.449275,-43.478261,78.260870
t1,0.0,0.0,1.0,-5.797101,-73.913043,89.043478
x2,0.0,1.0,0.0,-4.347826,-30.434783,34.782609
C - Z,0.0,0.0,0.0,-4.347826,-40.434783,54.782609



=== RESULT ===
Status: optimal
Objective value: 54.78260869565218
Solution: {'x1': np.float64(78.2608695652174), 'x2': np.float64(34.78260869565217)}
Iterations: 3


### โต๊ะ เก้าอี้(5)

In [ ]:
# Example (change these to your own problem)
sense = 'max'  # 'max' or 'min'
c = [60, 30, 20]     # maximize 3x1 + 2x2
A = [
    [8, 6, 1],
    [4, 2, 1.5],
    [2, 1.5, 0.5],
    [0, 1, 0]
    
]
b = [48, 20, 8, 5]
cons = ['<=', '<=', '<=', '<=']

Mode = 'TwoPhase'

run_and_show(sense, c, A, b, cons, Mode, M=1e5)


C:\Users\Admin\AppData\Local\Temp\ipykernel_2104\2753902525.py:172: RuntimeWarning: divide by zero encountered in divide
  ratios = np.where(mask, rhs / col, np.inf)


### Phase 2 — Initial C - Z row

,x1,x2,x3,s1,s2,s3,s4,RHS
s1,8.0,6.0,1.0,1.0,0.0,0.0,0.0,48.0
s2,4.0,2.0,1.5,0.0,1.0,0.0,0.0,20.0
s3,2.0,1.5,0.5,0.0,0.0,1.0,0.0,8.0
s4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,5.0
C - Z,60.0,30.0,20.0,0.0,0.0,0.0,0.0,0.0


Iteration 1 (Phase 2): Enter = x1, Leave = s3

### After pivot #1

...

### After pivot #2

,x1,x2,x3,s1,s2,s3,s4,RHS
s1,0.0,-2.00,0.0,1.0,2.0,-8.0,0.0,24.0
x3,0.0,-2.00,1.0,0.0,2.0,-4.0,0.0,8.0
x1,1.0,1.25,0.0,0.0,-0.5,1.5,0.0,2.0
s4,0.0,1.00,0.0,0.0,0.0,0.0,1.0,5.0
C - Z,0.0,-5.00,0.0,0.0,-10.0,-10.0,0.0,-280.0


### Phase 2 — Optimal (no positive C-Z). Objective = 280.000000

,x1,x2,x3,s1,s2,s3,s4,RHS
s1,0.0,-2.00,0.0,1.0,2.0,-8.0,0.0,24.0
x3,0.0,-2.00,1.0,0.0,2.0,-4.0,0.0,8.0
x1,1.0,1.25,0.0,0.0,-0.5,1.5,0.0,2.0
s4,0.0,1.00,0.0,0.0,0.0,0.0,1.0,5.0
C - Z,0.0,-5.00,0.0,0.0,-10.0,-10.0,0.0,-280.0



=== RESULT ===
Status: optimal
Objective value: 280.0
Solution: {'x1': np.float64(2.0), 'x2': 0.0, 'x3': np.float64(8.0)}
Iterations: 2


### ปุ๋ยเคมี NPK(6)

In [ ]:
# Example (change these to your own problem)
sense = 'min'  # 'max' or 'min'
c = [2, 3, 4]     # maximize 3x1 + 2x2
A = [
    [1, 1, 1],
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
    
]
b = [1000, 200, 400, 100]
cons = ['<=', '>=', '<=', '>=']

Mode = 'TwoPhase'

run_and_show(sense, c, A, b, cons, Mode, M=1e5)


C:\Users\Admin\AppData\Local\Temp\ipykernel_2104\2753902525.py:172: RuntimeWarning: divide by zero encountered in divide
  ratios = np.where(mask, rhs / col, np.inf)


### Phase 1 — Initial C - Z row

,x1,x2,x3,s1,t2,a2,s3,t4,a4,RHS
s1,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1000.0
a2,1.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,200.0
s3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,400.0
a4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,1.0,100.0
C - Z,1.0,0.0,1.0,0.0,-1.0,0.0,0.0,-1.0,0.0,300.0


Iteration 1 (Phase 1): Enter = x1, Leave = a2

### After pivot #1

...

### Phase 2 — Initial C - Z row

,x1,x2,x3,s1,t2,s3,t4,RHS
s1,0.0,1.0,0.0,1.0,1.0,0.0,1.0,700.0
x1,1.0,0.0,0.0,0.0,-1.0,0.0,0.0,200.0
s3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,400.0
x3,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,100.0
C - Z,0.0,-3.0,0.0,0.0,-2.0,0.0,-4.0,800.0


### Phase 2 — Optimal (no positive C-Z). Objective = -800.000000

,x1,x2,x3,s1,t2,s3,t4,RHS
s1,0.0,1.0,0.0,1.0,1.0,0.0,1.0,700.0
x1,1.0,0.0,0.0,0.0,-1.0,0.0,0.0,200.0
s3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,400.0
x3,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,100.0
C - Z,0.0,-3.0,0.0,0.0,-2.0,0.0,-4.0,800.0



=== RESULT ===
Status: optimal
Objective value: 800.0
Solution: {'x1': np.float64(200.0), 'x2': 0.0, 'x3': np.float64(100.0)}
Iterations: 2


### หน้า 18

In [ ]:
# Example (change these to your own problem)
sense = 'max'  # 'max' or 'min'
c = [60, 30, 20]     # maximize 3x1 + 2x2
A = [
    [8, 6, 1],
    [4, 2, 1.5],
    [2, 1.5, 0.5],
    [0,1, 0]
]
b = [48, 20, 8, 5]
cons = ['<=', '<=', '<=', '<=']

Mode = 'TwoPhase'

run_and_show(sense, c, A, b, cons, Mode, M=1e5)


C:\Users\Admin\AppData\Local\Temp\ipykernel_2104\2753902525.py:172: RuntimeWarning: divide by zero encountered in divide
  ratios = np.where(mask, rhs / col, np.inf)


### Phase 2 — Initial C - Z row

,x1,x2,x3,s1,s2,s3,s4,RHS
s1,8.0,6.0,1.0,1.0,0.0,0.0,0.0,48.0
s2,4.0,2.0,1.5,0.0,1.0,0.0,0.0,20.0
s3,2.0,1.5,0.5,0.0,0.0,1.0,0.0,8.0
s4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,5.0
C - Z,60.0,30.0,20.0,0.0,0.0,0.0,0.0,0.0


Iteration 1 (Phase 2): Enter = x1, Leave = s3

### After pivot #1

...

### After pivot #2

,x1,x2,x3,s1,s2,s3,s4,RHS
s1,0.0,-2.00,0.0,1.0,2.0,-8.0,0.0,24.0
x3,0.0,-2.00,1.0,0.0,2.0,-4.0,0.0,8.0
x1,1.0,1.25,0.0,0.0,-0.5,1.5,0.0,2.0
s4,0.0,1.00,0.0,0.0,0.0,0.0,1.0,5.0
C - Z,0.0,-5.00,0.0,0.0,-10.0,-10.0,0.0,-280.0


### Phase 2 — Optimal (no positive C-Z). Objective = 280.000000

,x1,x2,x3,s1,s2,s3,s4,RHS
s1,0.0,-2.00,0.0,1.0,2.0,-8.0,0.0,24.0
x3,0.0,-2.00,1.0,0.0,2.0,-4.0,0.0,8.0
x1,1.0,1.25,0.0,0.0,-0.5,1.5,0.0,2.0
s4,0.0,1.00,0.0,0.0,0.0,0.0,1.0,5.0
C - Z,0.0,-5.00,0.0,0.0,-10.0,-10.0,0.0,-280.0



=== RESULT ===
Status: optimal
Objective value: 280.0
Solution: {'x1': np.float64(2.0), 'x2': 0.0, 'x3': np.float64(8.0)}
Iterations: 2


### หน้า 20

In [ ]:
# Example (change these to your own problem)
sense = 'min'  # 'max' or 'min'
c = [0.4, 0.5]     # maximize 3x1 + 2x2
A = [
    [0.3, 0.1],
    [0.5, 0.5],
    [0.6, 0.4]

]
b = [2.7, 6, 6]
cons = ['<=', '=', '>=']

Mode = 'TwoPhase'

run_and_show(sense, c, A, b, cons, Mode, M=1e5)


### Phase 1 — Initial C - Z row

,x1,x2,s1,a2,t3,a3,RHS
s1,0.3,0.1,1.0,0.0,0.0,0.0,2.7
a2,0.5,0.5,0.0,1.0,0.0,0.0,6.0
a3,0.6,0.4,0.0,0.0,-1.0,1.0,6.0
C - Z,1.1,0.9,0.0,0.0,-1.0,0.0,12.0


Iteration 1 (Phase 1): Enter = x1, Leave = s1

### After pivot #1

,x1,x2,s1,a2,t3,a3,RHS
x1,1.0,0.333333,3.333333,0.0,0.0,0.0,9.0
a2,0.0,0.333333,-1.666667,1.0,0.0,0.0,1.5
a3,0.0,0.200000,-2.000000,0.0,-1.0,1.0,0.6
C - Z,0.0,0.533333,-3.666667,0.0,-1.0,0.0,2.1


...

### After pivot #4

,x1,x2,s1,t3,RHS
x1,1.0,0.0,5.0,0.0,7.50
t3,0.0,0.0,1.0,1.0,0.30
x2,0.0,1.0,-5.0,0.0,4.50
C - Z,0.0,0.0,-0.5,0.0,5.25


### Phase 2 — Optimal (no positive C-Z). Objective = -5.250000

,x1,x2,s1,t3,RHS
x1,1.0,0.0,5.0,0.0,7.50
t3,0.0,0.0,1.0,1.0,0.30
x2,0.0,1.0,-5.0,0.0,4.50
C - Z,0.0,0.0,-0.5,0.0,5.25



=== RESULT ===
Status: optimal
Objective value: 5.25
Solution: {'x1': np.float64(7.500000000000003), 'x2': np.float64(4.499999999999997)}
Iterations: 4


### เล่มบทที่ 2 

50

In [ ]:
# Example (change these to your own problem)
sense = 'min'  # 'max' or 'min'
c = [0.4, 0.5]     # maximize 3x1 + 2x2
A = [
    [0.3, 0.1],
    [0.5, 0.5],
    [0.6, 0.4]

]
b = [2.7, 6, 6]
cons = ['<=', '=', '>=']

Mode = 'TwoPhase'

run_and_show(sense, c, A, b, cons, Mode, M=1e5)


### Phase 1 — Initial C - Z row

,x1,x2,s1,a2,t3,a3,RHS
s1,0.3,0.1,1.0,0.0,0.0,0.0,2.7
a2,0.5,0.5,0.0,1.0,0.0,0.0,6.0
a3,0.6,0.4,0.0,0.0,-1.0,1.0,6.0
C - Z,1.1,0.9,0.0,0.0,-1.0,0.0,12.0


Iteration 1 (Phase 1): Enter = x1, Leave = s1

### After pivot #1

,x1,x2,s1,a2,t3,a3,RHS
x1,1.0,0.333333,3.333333,0.0,0.0,0.0,9.0
a2,0.0,0.333333,-1.666667,1.0,0.0,0.0,1.5
a3,0.0,0.200000,-2.000000,0.0,-1.0,1.0,0.6
C - Z,0.0,0.533333,-3.666667,0.0,-1.0,0.0,2.1


...

,x1,x2,s1,t3,RHS
x1,1.0,0.0,5.0,0.0,7.50
t3,0.0,0.0,1.0,1.0,0.30
x2,0.0,1.0,-5.0,0.0,4.50
C - Z,0.0,0.0,-0.5,0.0,5.25


### Phase 2 — Optimal (no positive C-Z). Objective = -5.250000

,x1,x2,s1,t3,RHS
x1,1.0,0.0,5.0,0.0,7.50
t3,0.0,0.0,1.0,1.0,0.30
x2,0.0,1.0,-5.0,0.0,4.50
C - Z,0.0,0.0,-0.5,0.0,5.25



=== RESULT ===
Status: optimal
Objective value: 5.25
Solution: {'x1': np.float64(7.500000000000003), 'x2': np.float64(4.499999999999997)}
Iterations: 4


: 

### BigM

In [ ]:
# Example (change these to your own problem)
sense = 'max'  # 'max' or 'min'
c = [3, 5]     # maximize 3x1 + 2x2
A = [
    [1, 0],
    [0, 2],
    [3, 2]

]
b = [4, 12, 18]
cons = ['<=', '<=', '=']

Mode = 'BigM'

run_and_show(sense, c, A, b, cons, Mode, M=1e5)

### Two-Phase Method

In [16]:
# Example (change these to your own problem)
sense = 'min'  # 'max' or 'min'
c = [0.4, 0.5]     # maximize 3x1 + 2x2
A = [
    [0.3, 0.1],
    [0.5, 0.5],
    [0.6, 0.4]
]
b = [2.7, 6, 6]
cons = ['<=', '=', '>=']

Mode = 'TwoPhase'

run_and_show(sense, c, A, b, cons, Mode, M=1e5)


=== Mode ===
Mode: TwoPhase


### Phase 1 — Initial C - Z row

,x1,x2,s1,a2,t3,a3,RHS
s1,0.3,0.1,1.0,0.0,0.0,0.0,2.7
a2,0.5,0.5,0.0,1.0,0.0,0.0,6.0
a3,0.6,0.4,0.0,0.0,-1.0,1.0,6.0
C - Z,1.1,0.9,0.0,0.0,-1.0,0.0,12.0


Iteration 1 (Phase 1): Enter = x1, Leave = s1

### After pivot #1

...

### After pivot #4

,x1,x2,s1,t3,RHS
x1,1.0,0.0,5.0,0.0,7.50
t3,0.0,0.0,1.0,1.0,0.30
x2,0.0,1.0,-5.0,0.0,4.50
C - Z,0.0,0.0,-0.5,0.0,5.25


### Phase 2 — Optimal (no positive C-Z). Objective = -5.250000

,x1,x2,s1,t3,RHS
x1,1.0,0.0,5.0,0.0,7.50
t3,0.0,0.0,1.0,1.0,0.30
x2,0.0,1.0,-5.0,0.0,4.50
C - Z,0.0,0.0,-0.5,0.0,5.25



=== RESULT ===
Status: optimal
Objective value: 5.25
Solution: {'x1': np.float64(7.500000000000003), 'x2': np.float64(4.499999999999997)}
Iterations: 4


# EX


## Example 2 — Two-Phase (with ">=" and "=")

This example requires artificial variables in Phase I.


In [8]:

# Maximize z = x1 + x2
# s.t.
#   -x1 +   x2 >= 1
#    x1 +   x2  = 3
#    x1, x2 >= 0
sense = 'max'
c = [1, 1]
A = [
    [-1, 1],
    [ 1, 1],
]
b = [1, 3]
cons = ['>=', '=']

Mode = 'TwoPhase'

run_and_show(sense, c, A, b, cons, Mode, M=1e5)


### Phase 1 — Initial C - Z row

,x1,x2,t1,a1,a2,RHS
a1,-1.0,1.0,-1.0,1.0,0.0,1.0
a2,1.0,1.0,0.0,0.0,1.0,3.0
C - Z,0.0,2.0,-1.0,0.0,0.0,4.0


Iteration 1 (Phase 1): Enter = x2, Leave = a1

### After pivot #1

,x1,x2,t1,a1,a2,RHS
x2,-1.0,1.0,-1.0,1.0,0.0,1.0
a2,2.0,0.0,1.0,-1.0,1.0,2.0
C - Z,2.0,0.0,1.0,-2.0,0.0,2.0


...

### Phase 2 — Initial C - Z row

,x1,x2,t1,RHS
x2,0.0,1.0,-0.5,2.0
x1,1.0,0.0,0.5,1.0
C - Z,0.0,0.0,0.0,-3.0


### Phase 2 — Optimal (no positive C-Z). Objective = 3.000000

,x1,x2,t1,RHS
x2,0.0,1.0,-0.5,2.0
x1,1.0,0.0,0.5,1.0
C - Z,0.0,0.0,0.0,-3.0



=== RESULT ===
Status: optimal
Objective value: 3.0
Solution: {'x1': np.float64(1.0), 'x2': np.float64(2.0)}
Iterations: 2



## Example 3 — Infeasible problem

This one has no feasible solution.


In [ ]:

# Maximize z = x1 + x2
# s.t.
#   x1 + x2 <= 1
#   x1 + x2 >= 3   (impossible simultaneously with <= 1)
#   x1, x2 >= 0
sense = 'max'
c = [1, 1]
A = [
    [1, 1],
    [1, 1],
]
b = [1, 3]
cons = ['<=', '>=']
Mode = 'TwoPhase'

run_and_show(sense, c, A, b, cons, Mode, M=1e5)


### Phase 1 — Initial C - Z row

,x1,x2,s1,t2,a2,RHS
s1,1.0,1.0,1.0,0.0,0.0,1.0
a2,1.0,1.0,0.0,-1.0,1.0,3.0
C - Z,1.0,1.0,0.0,-1.0,0.0,3.0


Iteration 1 (Phase 1): Enter = x1, Leave = s1

### After pivot #1

,x1,x2,s1,t2,a2,RHS
x1,1.0,1.0,1.0,0.0,0.0,1.0
a2,0.0,0.0,-1.0,-1.0,1.0,2.0
C - Z,0.0,0.0,-1.0,-1.0,0.0,2.0


### Phase 1 — Optimal (no positive C-Z). Objective = -2.000000

,x1,x2,s1,t2,a2,RHS
x1,1.0,1.0,1.0,0.0,0.0,1.0
a2,0.0,0.0,-1.0,-1.0,1.0,2.0
C - Z,0.0,0.0,-1.0,-1.0,0.0,2.0



=== RESULT ===
Status: infeasible
Objective value: nan
Solution: {}
Iterations: 1



## Example 4 — Unbounded problem


In [ ]:

# Maximize z = x1 + x2
# s.t.
#    x1 - x2 >= 0
#          x2 >= 0
# (No upper bound on x1, so objective tends to infinity)
sense = 'max'
c = [1, 1]
A = [
    [1, -1],
    [0,  1],
]
b = [0, 0]
cons = ['>=', '>=']

Mode = 'TwoPhase'

run_and_show(sense, c, A, b, cons, Mode, M=1e5)


C:\Users\Admin\AppData\Local\Temp\ipykernel_2104\2753902525.py:172: RuntimeWarning: invalid value encountered in divide
  ratios = np.where(mask, rhs / col, np.inf)


### Phase 1 — Initial C - Z row

,x1,x2,t1,a1,t2,a2,RHS
a1,1.0,-1.0,-1.0,1.0,0.0,0.0,0.0
a2,0.0,1.0,0.0,0.0,-1.0,1.0,0.0
C - Z,1.0,0.0,-1.0,0.0,-1.0,0.0,0.0


Iteration 1 (Phase 1): Enter = x1, Leave = a1

### After pivot #1

...

### Phase 2 — Initial C - Z row

,x1,x2,t1,t2,RHS
x1,1.0,0.0,-1.0,-1.0,0.0
x2,0.0,1.0,0.0,-1.0,0.0
C - Z,0.0,0.0,1.0,2.0,0.0


### Phase 2 — Unbounded in entering t1

,x1,x2,t1,t2,RHS
x1,1.0,0.0,-1.0,-1.0,0.0
x2,0.0,1.0,0.0,-1.0,0.0
C - Z,0.0,0.0,1.0,2.0,0.0



=== RESULT ===
Status: unbounded
Objective value: inf
Solution: {}
Iterations: 2
